# 🔬 Calculateur de doses - Études précliniques

Application simple pour calculer les besoins en produits pour vos études in vivo

In [ ]:
import pandas as pd
from ipywidgets import (
    IntSlider, FloatSlider, Text, Button, VBox, HBox, 
    FloatText, Dropdown, Output, HTML, Label
)
from IPython.display import display, Markdown, clear_output, HTML as DisplayHTML
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuration CSS pour une meilleure apparence
display(DisplayHTML("""
<style>
.widget-label { font-weight: bold; color: #333; }
.widget-output { margin: 10px 0; }
</style>
"""))

## ⚙️ Paramètres de l'étude

In [ ]:
# Paramètres globaux avec meilleur formatage
study_name = Text(
    value='Mon étude',
    placeholder='Nom de l\'étude',
    description='Nom de l\'étude:',
    style={'description_width': '180px'},
    layout={'width': '400px'}
)

n_mice = IntSlider(
    value=8,
    min=1,
    max=50,
    step=1,
    description='Souris par groupe:',
    style={'description_width': '180px'},
    layout={'width': '500px'}
)

weight = FloatSlider(
    value=20,
    min=1,
    max=100,
    step=1,
    description='Poids moyen (g):',
    style={'description_width': '180px'},
    layout={'width': '500px'}
)

duration = IntSlider(
    value=21,
    min=1,
    max=180,
    step=1,
    description='Durée (jours):',
    style={'description_width': '180px'},
    layout={'width': '500px'}
)

margin = FloatSlider(
    value=10,
    min=0,
    max=50,
    step=5,
    description='Marge de sécurité (%):',
    style={'description_width': '180px'},
    layout={'width': '500px'}
)

info_margin = Label(value='💡 Conseil: 10-20% pour compenser les pertes de préparation')

params_box = VBox([
    study_name,
    n_mice,
    weight,
    duration,
    margin,
    DisplayHTML('<p style="color: #0066cc; font-size: 12px; margin-top: 10px;">💡 Conseil: Ajoutez 10-20% de marge pour compenser les pertes</p>')
])

display(params_box)

## 📋 Définition des groupes

In [ ]:
n_groups = IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description='Nombre de groupes:',
    style={'description_width': '180px'},
    layout={'width': '500px'}
)

display(n_groups)

In [ ]:
# Initialisation globale
groups_list = []
groups_output = Output()

def create_group_widgets(n):
    groups_widgets = []
    
    for i in range(n):
        group_name = Text(
            value='Groupe ' + str(i+1),
            description='Nom:',
            style={'description_width': '100px'},
            layout={'width': '300px'}
        )
        
        dose = FloatText(
            value=0,
            min=0,
            max=1000,
            step=5,
            description='Dose (mg/kg):',
            style={'description_width': '130px'},
            layout={'width': '250px'}
        )
        
        dosing = Dropdown(
            options=[('QD (Quotidien)', 1), ('BID (2x/jour)', 0.5)],
            value=1,
            description='Dosing:',
            style={'description_width': '100px'}
        )
        
        group_box = VBox([
            DisplayHTML('<h4 style="color: #0066cc; margin: 15px 0 10px 0;">📌 Groupe ' + str(i+1) + '</h4>'),
            HBox([group_name, dose, dosing]),
        ])
        
        groups_widgets.append({
            'name': group_name,
            'dose': dose,
            'dosing': dosing,
            'box': group_box
        })
    
    return groups_widgets

def update_groups(change=None):
    global groups_list
    n = n_groups.value
    groups_list = create_group_widgets(n)
    
    with groups_output:
        clear_output()
        display(VBox([g['box'] for g in groups_list]))

n_groups.observe(update_groups, names='value')
update_groups()
display(groups_output)

## 📊 Résultats

In [ ]:
def calculate_doses():
    if not groups_list:
        return None
    
    wt_kg = weight.value / 1000
    results = []
    
    for idx, group in enumerate(groups_list, 1):
        group_name = group['name'].value
        dose = group['dose'].value
        dosing_freq = group['dosing'].value
        
        n_doses = duration.value / dosing_freq
        total_dose = dose * wt_kg * n_mice.value * n_doses
        total_dose_margin = total_dose * (1 + margin.value/100)
        
        dosing_label = 'QD' if dosing_freq == 1 else 'BID'
        margin_int = int(margin.value)
        
        results.append({
            'Groupe': 'G' + str(idx) + ': ' + group_name,
            'Dose (mg/kg)': dose,
            'Dosing': dosing_label,
            'Composé (mg)': round(total_dose, 2),
            'Composé +' + str(margin_int) + '% (mg)': round(total_dose_margin, 2)
        })
    
    df = pd.DataFrame(results)
    return df

calc_button = Button(
    description='🧮 CALCULER LES QUANTITÉS',
    button_style='success',
    tooltip='Lance le calcul des doses',
    layout={'width': '300px', 'height': '40px'},
    font_size='14'
)

results_output = Output()

def on_calc_button_clicked(b):
    with results_output:
        clear_output()
        df = calculate_doses()
        
        if df is not None:
            study = study_name.value
            
            # En-tête
            display(DisplayHTML('<hr>'))
            display(DisplayHTML('<h3 style="color: #333;">✅ Résultats pour: <b style="color: #0066cc;">' + study + '</b></h3>'))
            
            # Tableau avec meilleur formatage
            df_display = df.copy()
            display(df_display)
            
            # Métriques principales
            margin_int = int(margin.value)
            total_col = 'Composé +' + str(margin_int) + '% (mg)'
            total = df[total_col].sum()
            total_without_margin = df['Composé (mg)'].sum()
            
            display(DisplayHTML('<hr>'))
            display(DisplayHTML(
                '<h3 style="text-align: center;">📦 Quantité totale à commander</h3>' +
                '<div style="display: flex; justify-content: center; gap: 40px; margin: 20px 0;">' +
                '<div style="text-align: center; padding: 15px; background: #f0f0f0; border-radius: 8px;">' +
                '<p style="margin: 0; color: #666; font-size: 12px;">Sans marge</p>' +
                '<p style="margin: 5px 0 0 0; font-size: 24px; font-weight: bold; color: #0066cc;">' + str(round(total_without_margin, 1)) + ' mg</p>' +
                '</div>' +
                '<div style="text-align: center; padding: 15px; background: #e8f4f8; border-radius: 8px; border: 2px solid #0066cc;">' +
                '<p style="margin: 0; color: #0066cc; font-size: 12px; font-weight: bold;">Avec marge (+' + str(margin_int) + '%)</p>' +
                '<p style="margin: 5px 0 0 0; font-size: 28px; font-weight: bold; color: #0066cc;">' + str(round(total, 1)) + ' mg</p>' +
                '</div>' +
                '</div>'
            ))
            
            # Boutons d'export
            export_csv_btn = Button(
                description='📥 Télécharger CSV',
                button_style='info',
                tooltip='Exporte les résultats en CSV',
                layout={'width': '180px'}
            )
            
            export_excel_btn = Button(
                description='📥 Télécharger Excel',
                button_style='info',
                tooltip='Exporte les résultats en Excel',
                layout={'width': '180px'}
            )
            
            def export_csv(b):
                study_clean = study_name.value.replace(' ', '_')
                date_str = datetime.now().strftime('%Y%m%d')
                filename = 'doses_' + study_clean + '_' + date_str + '.csv'
                df.to_csv(filename, index=False)
                print('✅ Fichier exporté: ' + filename)
            
            def export_excel(b):
                study_clean = study_name.value.replace(' ', '_')
                date_str = datetime.now().strftime('%Y%m%d')
                filename = 'doses_' + study_clean + '_' + date_str + '.xlsx'
                df.to_excel(filename, index=False, sheet_name='Doses')
                print('✅ Fichier exporté: ' + filename)
            
            export_csv_btn.on_click(export_csv)
            export_excel_btn.on_click(export_excel)
            
            display(DisplayHTML('<h4>Exporter les résultats</h4>'))
            display(HBox([export_csv_btn, export_excel_btn]))
            display(DisplayHTML('<hr>'))

calc_button.on_click(on_calc_button_clicked)
display(DisplayHTML('<br>'))
display(calc_button)
display(results_output)

## ℹ️ Information

In [ ]:
display(DisplayHTML("""
<div style="background: #f9f9f9; padding: 15px; border-left: 4px solid #0066cc; margin-top: 20px;">
<h4 style="margin-top: 0; color: #0066cc;">📖 Informations de calcul</h4>
<p><b>QD (Once Daily)</b>: 1 dose par jour</p>
<p><b>BID (Twice Daily)</b>: 2 doses par jour</p>
<p style="margin-bottom: 0;"><b>Formule</b>: Total = Dose (mg/kg) × Poids (kg) × Nombre d'animaux × Nombre de doses</p>
</div>
"""))